In [1]:
import deepspeech
import numpy as np
import wave
from pydub import AudioSegment
import os


In [1]:
from pydub import AudioSegment

# Load the audio file (replace "full/path/to/hello_example.m4a" with the actual path)
audio = AudioSegment.from_file("hello_example.m4a")

# Convert audio to the required format
audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)

# Export the audio to WAV format
audio.export("hello.wav", format='wav')

<_io.BufferedRandom name='hello.wav'>

In [2]:
import deepspeech
import numpy as np
import wave
from pydub import AudioSegment

# Convert audio to required format
def convert_audio(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(output_file, format='wav')

# Read the WAV file
def read_wav_file(filename):
    with wave.open(filename, 'rb') as wf:
        sample_rate = wf.getframerate()
        assert sample_rate == 16000, 'Only 16000Hz input WAV files are supported.'
        audio = np.frombuffer(wf.readframes(wf.getnframes()), dtype=np.int16)
    return audio

# Absolute paths to model and scorer files
model_file_path = ".deepspeech-model/deepspeech-0.9.3-models.pbmm"
scorer_file_path = ".deepspeech-model/deepspeech-0.9.3-models.scorer"

# Check if model and scorer files are accessible
try:
    with open(model_file_path, 'rb') as f:
        pass
    with open(scorer_file_path, 'rb') as f:
        pass
except Exception as e:
    print(f"Error accessing model or scorer file: {e}")
    raise

# Load the model and scorer
model = deepspeech.Model(model_file_path)
model.enableExternalScorer(scorer_file_path)

# Paths to your audio files
input_audio_file = ".audio_files/input/trialv.m4a"
converted_audio_file = ".audio_files/output/trialv.wav"

# Convert and transcribe
convert_audio(input_audio_file, converted_audio_file)
audio_data = read_wav_file(converted_audio_file)
transcribed_text = model.stt(audio_data)

print('Transcribed Text:', transcribed_text)

Transcribed Text: this is the reference passage that the use of wire at the indiscreet and accurate


In [4]:
# Reference passage
reference_passage = "This is the reference passage that the user will read. Ensure it is clear and accurate."

# Function to compare the transcribed text with the reference passage
def analyze_pronunciation(reference, transcribed):
    ref_words = reference.split()
    trans_words = transcribed.split()
    
    feedback = []
    correct_count = 0
    total_count = len(ref_words)
    
    for i, ref_word in enumerate(ref_words):
        if i < len(trans_words):
            trans_word = trans_words[i]
            if ref_word.lower() == trans_word.lower():
                correct_count += 1
            else:
                feedback.append(f'Expected "{ref_word}", but got "{trans_word}" at word position {i + 1}.')
        else:
            feedback.append(f'Missing word "{ref_word}" at position {i + 1}.')
    
    if len(trans_words) > len(ref_words):
        for j in range(len(ref_words), len(trans_words)):
            feedback.append(f'Extra word "{trans_words[j]}" at position {j + 1}.')
    
    accuracy = (correct_count / total_count) * 100
    return accuracy, feedback

# Analyze pronunciation and calculate accuracy
accuracy, feedback = analyze_pronunciation(reference_passage, transcribed_text)

# Provide feedback
print(f"Pronunciation Accuracy: {accuracy:.2f}%")
print("Feedback on Pronunciation:")
for msg in feedback:
    print(msg)

Pronunciation Accuracy: 43.75%
Feedback on Pronunciation:
Expected "user", but got "use" at word position 8.
Expected "will", but got "of" at word position 9.
Expected "read.", but got "wire" at word position 10.
Expected "Ensure", but got "at" at word position 11.
Expected "it", but got "the" at word position 12.
Expected "is", but got "indiscreet" at word position 13.
Expected "clear", but got "and" at word position 14.
Expected "and", but got "accurate" at word position 15.
Missing word "accurate." at position 16.
